In [99]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import re

In [100]:
# Create DataFrames from CSV file
t0 = time.time()
nlp_df = pd.read_csv('../../Data/nlp_df.csv')
lyric_TF_df = pd.read_csv('../../Data/lyric_TF.csv').drop([0])
lyric_TF_df.index = lyric_TF_df.index - 1
top_cooccurring_TF_df = pd.read_csv('../../Data/top_cooccurring_TF.csv')
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
display(nlp_df.head(1))
display(lyric_TF_df.head(3))
top_cooccurring_TF_df

Run time: 253.83659100532532 seconds


,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['', 'im', 'like', 'water', 'ship', 'rolled', ...","(1, 'en')"


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,sufficient,girly,reek,duffel,apartheid,unrestrained,bitter,staff,wellve,eighth
0,NaN,NaN,NaN,53468.0,NaN,4528.6433,5362.4272,42651.0,-60641.618,5517.0,...,4.0,14.0,6.0,13.0,2.0,2.0,143.0,12.0,2.0,5.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,oh oh,yeah yeah,ooh ooh,na na,dont know,la la,dont wanna,let go,im gonna,oh yeah,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,4776,3233,2573,2425,2384,2343,1519,1238,1208,1075,...,260,258,257,257,256,255,254,253,251,250


In [101]:
# Create a list of all words, unique word counts, and filtered words
t0 = time.time()
words_list = []
unique_word_counts = []
filtered_words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    filtered_words = filtered_words.replace(',', '').replace("'", '')
    filtered_words = filtered_words.replace('[', '').replace(']', '')
    filtered_words = filtered_words.replace('#', '').replace('&nbsp', '')
    filtered_words = filtered_words.replace('?', '? ').replace(',', '')
    filtered_words = filtered_words.replace('/', ' ')
    filtered_words = filtered_words.replace('\\u200a', '').replace('\\u200b', '')
    filtered_words = filtered_words.replace('\\u2063', '').replace('\u202f', '')
    filtered_words = filtered_words.replace('\\u2028', ' ').replace('\\u2008', ' ')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    unique_words = list(set(filtered_words.strip().split(' ')))
    unique_word_counts.append(len(unique_words))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(word_columns)

Run time: 22.85764193534851 seconds


36234

In [102]:
# Add a unique_word_count column and replace the filtered column
nlp_df['unique_word_count'] = unique_word_counts
nlp_df['filtered'] = filtered_words_list
nlp_df.head(1)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,filtered,language,unique_word_count
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,im like water ship rolled night rough surface...,"(1, 'en')",87


In [103]:
# Remove songs with less than 25 unique words
nlp_df = nlp_df[nlp_df['unique_word_count']>=25]
len(nlp_df)

8041

In [117]:
# Add co-occurring word columns
cooccurring_words = list(top_cooccurring_TF_df.columns)
for words in cooccurring_words:
    lyric_TF_df[words] = 0
len(lyric_TF_df.columns)

21888

In [118]:
# Add the term frequencies to the DataFrame
t0 = time.time()
nlp_df = nlp_df.reset_index()
uncommon_cooccurring_words = []
for index, row in lyric_TF_df.iterrows():
    lyrics = nlp_df['filtered'][index].strip().split(' ')
    while '' in lyrics:
        lyrics.remove('')
    while ' ' in lyrics:
        lyrics.remove(' ')
    for i in range(len(lyrics)-1):
        co_occurring_words = lyrics[i] + ' ' + lyrics[i+1]
        if co_occurring_words in cooccurring_words:
            lyric_TF_df.loc[index, co_occurring_words]+=1
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
lyric_TF_df.head(3)

Run time: 139.1545763015747 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,-7.357,1.0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
# Create new total row
lyric_TF_df.loc[-1] = lyric_TF_df.sum(numeric_only=True)
lyric_TF_df.index = lyric_TF_df.index + 1  
lyric_TF_df = lyric_TF_df.sort_index()
lyric_TF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,NaN,NaN,NaN,53468.0,NaN,4528.6433,5362.4272,42651.0,-60641.618,5517.0,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,240.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
# Create filtered DataFrame without columns of words appear less than 4 times
filtered_lyric_TF_df = lyric_TF_df.drop(columns=lyric_TF_df.columns[(lyric_TF_df[0:1]<4).any()])    
filtered_lyric_TF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,NaN,NaN,NaN,53468.0,NaN,4528.6433,5362.4272,42651.0,5517.0,665.7547,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,240.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
# Save DataFrames to CSV
t0 = time.time()
filtered_lyric_TF_df.to_csv('../../Data/filtered_cooccurring_TF.csv', index=False)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

t0 = time.time()
lyric_TF_df.to_csv('../../Data/cooccurring_TF.csv', index=False)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 163.83235478401184 seconds
Run time: 297.3798201084137 seconds


In [ ]:
# Make the song name be the index of the DataFrame